In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import os
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import torch
import sklearn.metrics
PATH = Path('../input')
TRAIN = Path('../input/train')
MODELS = Path('./')
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

# **Looking at Classes**

In [ ]:
os.makedirs('Models', exist_ok=True)
os.listdir('.')

In [ ]:
classes = !ls {TRAIN}
print('Number of Samples per class :')
for i in classes:
    print(i,':', len(get_image_files(TRAIN/classes[0])))

In [ ]:
# Looking at one random image
im_path = get_image_files(TRAIN/classes[0])[np.random.randint(0, 263, 1)[0]]
img = cv2.imread(str(im_path))
print(img.shape)
plt.imshow(img)

We see that the classes are equally distriuted

In [ ]:
tfms = get_transforms(flip_vert=True, max_rotate=90, max_zoom=1.3)
bs = 64
sz = 224
nw = 0
data = ImageDataBunch.from_folder(TRAIN, test=PATH/'test', valid_pct=0.2, ds_tfms=tfms, bs=bs, size=sz, num_workers=nw).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3)

In [ ]:
data.classes

In [ ]:
def f1_micros(y_true, y_pred):
    return sklearn.metrics.fl_score(y_true, y_pred, average='micro')
sklearn.metrics.f1_score([1,0,1], [1,0,1])

In [ ]:
fl_micros([1,1,1], [1,1,0])

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=f1_micro, path=MODELS)
learn.load('init_model')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-2
learn.fit(1, lr)

In [ ]:
learn.save('init_model')

In [ ]:
os.listdir('.')